# Uso de RDD's avanzado
En Big Data es habitual trabajar con datos en formato clave-valor. Por ello, Spark ofrece transformaciones y acciones específicas para estos casos

In [ ]:
# Inicializamos SparkSession y SparkContext
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("spark://spark-master:7077") \
    .appName("02-rdd2") \
    .config("spark.eventLog.enabled", "true") \
    .config("spark.eventLog.dir", "hdfs:///spark/logs/history") \
    .config("spark.history.fs.logDirectory", "hdfs:///spark/logs/history") \
    .getOrCreate()

spark.version  # Verifica la versión de Spark

#spark = SparkSession.builder.getOrCreate()

sc = spark.sparkContext

In [ ]:
rdd_pares1 = sc.parallelize([('a', 1), ('b', 1), ('c', 1)])
print (rdd_pares1.collect())

rdd_st = sc.parallelize ("Big Data aplicado. Curso de especialización de Inteligencia Artificial y Big Data".split())
rdd_pares2 = rdd_st.map(lambda palabra: (palabra,1))
print (rdd_pares2.collect())

Lo interesante de los RDD's de pares clave valor es que proporcionan una serie de transformaciones y acciones adicionales
## Operaciones clave-valor básicas
### keyBy
Función que crea una clave para cada valor actual de un RDD

In [ ]:
# La clave es la inicial de cada palabra
rdd_pares = rdd_st.keyBy(lambda palabra: palabra[0])
print(rdd_pares.collect())
rdd_pares2 = rdd_st.keyBy (lambda palabra: len(palabra))
print (rdd_pares2.collect())

### mapValues
Realiza una operación map sólo sobre los valores del RDD

In [ ]:
rdd_pares.mapValues(lambda x: x.upper()).collect()

### keys
Devuelve un RDD sólo con las claves

In [ ]:
rdd_pares.keys().collect()

### values
Devuelve un RDD sólo con los valores

In [ ]:
rdd_pares.values().collect()

### lookup
Devuelve sólo los valores que coinciden con la clave especificada

In [ ]:
rdd_pares.lookup('B')

### sampleByKey

In [ ]:
caracteres_distintos = rdd_pares.keys().collect()
map_muestra = dict(map(lambda c: (c, random.random()), caracteres_distintos))
rdd_pares.sampleByKey(True, map_muestra, 6).collect()


## Agregaciones
### groupByKey
Agrupa los valores en función de la clave

In [ ]:
for key, values_iterable in rdd_pares.groupByKey().collect():
    # Convertir el iterable de resultados a una lista
    values_list = list(values_iterable)
    # Imprimir la clave y los valores
    print(f"Clave: {key}, Valores: {values_list}")

### reduceByKey
Aplica una función reductora después de agrupar los valores del RDD en función de la clave

In [ ]:
rdd_pares.reduceByKey(lambda x,y:len(x)+len(y)).collect()

### sortByKey
Ordena

In [ ]:
rdd_pares3.sortByKey(ascending=False).collect()

### countByKey
Permite contar el número de valores que se corresponden con una determinada clave

In [ ]:
rdd_pares.countByKey()

### aggregate
Esta función requiere un *null* y un *valor inicial*, así como dos funciones. La primera agrega dentro de una partición y la segunda entre particiones:

In [ ]:
rdd_pruebas = sc.parallelize([1,2,3,4],2)

print(rdd_pruebas.glom().collect())
max_func = (lambda x,y: (x[0] +y, x[1] +1))
add_func = (lambda x,y: (x[0] + y[0], x[1] + y[1]))
rdd_pruebas.aggregate((0,0),max_func, add_func)

In [ ]:
seq_op = (lambda x,y: (x[0] * y, x[1]+1))
comb_op = (lambda x,y: (x[0] * y[0], x[1]+y[1]))
rdd_pruebas.aggregate((1,0),seq_op, comb_op)

In [ ]:
max_func = (lambda x,y: x+y)
add_func = (lambda x,y: x+y)
rdd_pruebas.aggregate(0,max_func, add_func)

### aggregateByKey
Similar a *aggregate* pero en vez de hacer la agregación partición a partición lo hace clave a clave.

In [ ]:
# Ejemplo aggregateByKey
rdd_ejemplo2 = sc.parallelize([("a",1),("a",2),("a",3),("b",4),("b",5)])
seq_op = (lambda acc, new_value: (acc[0]+new_value, acc[1]+1))
comb_op = (lambda r1, r2: (r1[0] + r2[0], r1[1]+r2[1]))
rdd_ejemplo2.aggregateByKey((0,0), seq_op, comb_op).collect()

### combineByKey
Permite combinar valores y distribuirlos en número especificado de particiones. Necesita los siguiente parámetros:
- Función valor a combinable: Mapea los valores a valores combinables (ejemplo: entero a array)
- Función mezclar valores: Mezcla los valores (ejemplo: añade valores a un array)
- Función mezclar combinaciones: Une los resultados de las distintas particiones (ejemplo: une los arrays)

In [ ]:
def valor_a_comb (valor):
    return [valor]

def mezclar_valores_func (valores, valor_nuevo):
    valores.append(valor_nuevo)
    return valores

def mezclar_comb_func (valores1, valores2):
    return valores1 + valores2

particiones_salida = 3
rdd_ejemplo2.combineByKey(valor_a_comb,mezclar_valores_func, mezclar_comb_func,particiones_salida).collect()



### foldByKey
Mezcla los valores para cada clave usando una función asociativa y un *"valor neutral"*. El resultado de dicha función debe ser del mismo tipo que los valores.

In [ ]:
seq_op = (lambda acc, new_value: acc+new_value)
rdd_ejemplo2.foldByKey(0,seq_op).collect()

### coGroup
Permite agrupar hasta dos RDD's clave-valor. El resultado es un nuevo RDD clave valor donde el *valor* es un array formado por los valores de esa clave en ambos RDD's originales

In [ ]:
import random
distinctChars = rdd_st.flatMap(lambda word: word.lower()).distinct()
charRDD = distinctChars.map(lambda c: (c, random.random()))
charRDD2 = distinctChars.map(lambda c: (c, random.random()))
resultado = charRDD.cogroup(charRDD2).take(5)

for clave, valores in resultado:
    print("Clave:", clave)
    print("Valores de charRDD:", list(valores[0]))  # Convertimos el iterable de valores a lista
    print("Valores de charRDD2:", list(valores[1])) # Convertimos el iterable de valores a lista

## joins
Hay varios tipos posibles de join
- *Inner join*: join()
- *Full outer join*: fullOuterJoin()
- *left outer join*: leftOuterJoin()
- *right outer join*: rightOuterJoin()
- *producto cartesiano*: cartesian() (no se recomienda su uso)

In [ ]:
rdd1 = sc.parallelize([('a', 1), ('b', 2), ('c', 3), ('d', 4)])
rdd2 = sc.parallelize([('a', 4), ('b', 5), ('c', 6), ('e', 5)])
rdd3 = rdd1.join(rdd2)
print("Inner join:")
print(rdd3.collect())

rdd4 = rdd1.fullOuterJoin(rdd2)
print ("Full Outer join:")
print (rdd4.collect())

rdd5 = rdd1.leftOuterJoin(rdd2)
print ("Left Outer join:")
print (rdd5.collect())

rdd6 = rdd1.rightOuterJoin(rdd2)
print ("Right Outer join:")
print (rdd6.collect())


rdd7 = rdd1.cartesian(rdd2)
print ("Cartesian:")
print (rdd7.collect())

### union
Permite unir varios RDD's

In [ ]:
rdda = sc.parallelize ([('a',1),('b',2),('b',3)])
rddb = sc.parallelize ([('a',3),('b',1),('c',2)])
rdd_union = rdda.union(rddb)
print(rdd_union.collect())
# Se puede combinar con reduceByKey()
print(rdd_union.reduceByKey(lambda x,y: x+y).collect())

### zip
No es exactamente un join pero sirve para unir dos RDD's. En este caso, asume que ambos tienen la misma longitud y crea un RDD clave-valor, donde la clave es un elemento del primer RDD y el valor uno del segundo

In [ ]:
rdd1 = sc.parallelize(range(10),3)
rdd2 = sc.parallelize("Esta es una frase de prueba a ver que tal".split(),3)
rdd1.zip(rdd2).collect()

## Controlando particiones
La API de RDD's permite controlar cómo se distribuyen físicamente los datos a través del cluster.
### coalesce
Permite *colapsar* las particiones que se encuentran en el mismo *worker* con el objetivo de evitar *barajar* los datos al reparticionar.

In [ ]:
print (rdd_st.getNumPartitions())
print (rdd_st.coalesce(2).getNumPartitions())

### repartition
Permite modificar el número de particiones, pero realiza un proceso de *baraje* a través de los nodos.

In [ ]:
print (rdd_st.repartition(4).getNumPartitions())

### repartitionAndSortWithinPartitions
Permite reparticionar y, además, ordenar los valores dentro de cada partición. Admite los siguientes parámetros:
- *num_particiones* ( *numPartitions*): Opcional. Permite establecer el número de particiones
- *funcion_particionamiento* (*partitionFunc*): Opcional. Permite controlar el índice de particiones
- *ascendente* (*ascending*): Opcional. Permite establecer si el ordenamiento es ascendente (por defecto) o descendente.
- *funcion_clave* (*keyfunc*): Permite realizar cálculos sobre la clave


In [ ]:

rdd = sc.parallelize([(0, 5), (3, 8), (2, 6), (0, 8), (3, 8), (1, 3)])
rdd2 = rdd.repartitionAndSortWithinPartitions(2, lambda x: x % 2, True)
rdd2.glom().collect()
